# 13.2 Patsy를 이용해서 모델 생성하기

In [2]:
# Patsy(팻시)는 통계 모델(특히 선형 모델)을 위한 파이썬 라이브러리이다.  
# Patsy는 R이나 S 통계 프로그래밍 언어에서 사용하는 수식 문법과 비슷한 형식의 문자열 기반 "수식 문법"을 제공한다.

In [3]:
# Patsy는 통계 모델에서 선형 모델을 잘 지원하므로 이해를 돕기 위해 주요 기능 중 일부만 살펴보도록 하자.
# Patsy의 수식 문법은 다음과 같은 특수한 형태의 문자열이다.

    y ~ x0 + x1

In [4]:
# a + b 문법은 a와 b를 더하라는 의미가 아니라 모델을 위해 생성된 배열을 설계하는 용법이다.
# patsy.dmatrices 함수는 수식 문자열과 데이터셋(DataFrame 또는 배열의 사전)을 함께 받아 선형 모델을 위한 설계 배열을 만들어낸다.

In [5]:
import pandas as pd
import numpy as np

In [6]:
data = pd.DataFrame({
    "x0": [1, 2, 3, 4, 5],
    "x1": [0.01, -0.01, 0.25, -4.1, 0.],
    "y": [-1.5, 0., 3.6, 1.3, -2.]})

In [7]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [8]:
import patsy

In [9]:
y, X = patsy.dmatrices("y ~ x0 + x1", data)

In [10]:
# dmatrices 함수를 실행하면 다음과 같은 결과를 얻을 수 있다.

In [11]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [12]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [13]:
# Patsy의 DesignMatrix 인스턴스는 몇 가지 추가 데이터가 포함된 NumPy ndarray로 볼 수 있다.

In [14]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [15]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [16]:
# 여기서 Intercept는 최소자승회귀와 같은 선형 모델을 위한 표현이다.
# 모델에 0을 더해서 intercept(절편)를 제거할 수 있다.

In [17]:
patsy.dmatrices("y ~ x0 + x1 + 0", data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [18]:
# Patsy 객체는 최소자승회귀분석을 위해 numpy.linalg.lstsq 같은 알고리즘에 바로 넘길 수도 있다.

In [20]:
coef, resid, _, _ = np.linalg.lstsq(X, y)

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_24348\2525922789.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid, _, _ = np.linalg.lstsq(X, y)


In [22]:
# 모델 메타데이터는 design_info 속성을 통해 얻을 수 있는데 예를 들면 모델의 컬럼명을 피팅된 항에 맞추어 Series를 만들어낼 수도 있다.

In [23]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [24]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)

In [25]:
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

- 13.2.1 Patsy 용법으로 데이터 변환하기

In [26]:
# 파이썬 코드를 Patsy 용법과 섞어서 사용할 수도 있는데, Patsy 문법을 해석하는 과정에서 해당 함수를 찾아 실행해준다.

In [27]:
y, X = patsy.dmatrices("y ~ x0 + np.log(np.abs(x1) + 1)", data)

In [28]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [29]:
# 자주 쓰이는 변수 변환으로는 표준화(평균 0, 분산 1)와 센터링(평균값을 뺌)이 있는데 Patsy에는 이런 목적을 위한 내장 함수가 존재한다.

In [30]:
y, X = patsy.dmatrices("y ~ standardize(x0) + center(x1)", data)

In [31]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [32]:
# 모델링 과정에서 모델을 어떤 데이터셋에 피팅하고 그다음에 다른 모델에 기반하여 평가해야 하는 경우가 있다.
# 이는 홀드-아웃이거나 신규 데이터가 나중에 관측되는 경우다.
# 센터링이나 표준화 같은 변환을 적용하는 경우 새로운 데이터에 기반하여 예측하기 위한 용도로 모델을 사용한다면 주의해야 한다.
# 이를 상태를 가지는 변환(stateful)이라고 하는데 새로운 데이터셋을 변경하기 위해 원본 데이터의 표준편차나 평균 같은 통계를 사용해야 하기 때문이다.

In [33]:
# patsy.build_design_matrices 함수는 입력으로 사용되는 원본 데이터셋에서 저장한 정보를 사용해서 출력 데이터를 만들어내는 변환에 적용할 수 있는 함수다.

In [34]:
new_data = pd.DataFrame({
    "x0": [6, 7, 8, 9],
    "x1": [3.1, -0.5, 0, 2.3], 
    "y": [1, 2, 3, 4]})

In [35]:
new_X = patsy.build_design_matrices([X.design_info], new_data)

In [36]:
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [37]:
# Patsy 문법에 더하기 기호(+)는 덧셈이 아니므로 데이터셋에서 이름으로 컬럼을 추가하고 싶다면 I라는 특수한 함수로 둘러싸야 한다.

In [38]:
y, X = patsy.dmatrices("y ~ I(x0 + x1)", data)

In [39]:
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

In [40]:
# Patsy는 patsy.builtins 모듈 내에 여러 가지 변환을 위한 내장 함수들을 제공한다.

- 13.2.2 범주형 데이터와 Patsy

In [41]:
# 비산술 데이터는 여러 가지 형태의 모델 설계 배열로 변환될 수 있다.
# Patsy에서 비산술 용법을 사용하면 기본적으로 더미 변수로 변환된다.
# 만약 intercept가 존재한다면 공선성을 피하기 위해 레벨 중 하나는 남겨두게 된다.

In [42]:
data = pd.DataFrame({
    "key1": ["a", "a", "b", "b", "a", "b", "a", "b"],
    "key2": [0, 1, 0, 1, 0, 1, 0, 0],
    "v1"  : [1, 2, 3, 4, 5, 6, 7, 8],
    "v2"  : [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]    
})

In [43]:
y, X = patsy.dmatrices("v2 ~ key1", data)

In [44]:
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [45]:
# 모델에서 intercept를 생략하면 각 범주값의 컬럼은 모델 설계 배열에 포함된다.

In [46]:
y, X = patsy.dmatrices("v2 ~ key1 + 0", data)

In [47]:
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [48]:
# 산술 컬럼은 C 함수를 이용해서 범주형으로 해석할 수 있다.

In [49]:
y, X = patsy.dmatrices("v2 ~ C(key2)", data)

In [50]:
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [51]:
# 모델에서 여러 범주형 향을 사용한다면 ANOVA(분산 분석) 모델에서처럼 key1:key2 같은 용법을 사용할 수 있게 되므로 더 복잡해진다.

In [52]:
data["key2"] = data["key2"].map({0: "zero", 1: "one"})

In [53]:
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [54]:
y, X = patsy.dmatrices("v2 ~ key1 + key2", data)

In [55]:
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [56]:
y, X = patsy.dmatrices("v2 ~ key1 + key2 + key1:key2", data)

In [57]:
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

In [58]:
# Patsy는 특정 순서에 따라 데이터를 변환하는 방법을 포함하여 범주형 데이터를 변환하는 여러 가지 방법을 제공한다.